In [1]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
import asyncio
import pandas as pd
import numpy as np
import datetime

import stream.metricspandas as metricpandas
import stream.stream as stream

SERVER="nats://dbs001.edgefarm.dev:4222"
EXPORT_SUBJECT = "agilis-mc-1.EXPORT.metrics"
STREAM_NAME = "bogie-network_export-stream-aggregate"
NATS_CREDS = "/home/klaus/work/ops.edgefarm-deployments/overlays/edgefarm.dev/dbs001/source/nsc/nkeys/creds/edgefarm_operator/acc3/user.creds"

In [9]:

async def ns_from_time(t):
    ns = await stream.NatsStream.from_start_time(
        SERVER, NATS_CREDS, STREAM_NAME, EXPORT_SUBJECT, t
    )
    # "1990-01-01T00:00:00.000000Z"
    return ns

async def nats_stream_fetch(ns, n):
    print("nats_stream_fetch %d" % n)
    df = pd.DataFrame()
    for i in range(n):
        msg = await ns.next_msg(timeout=10.0)
        if msg is None:
            print("msg is None")
            break
        await ns.ack(msg)
        df2 = metricpandas.metrics_nats_to_pandas(msg)
        print("loaded time %s seq %d" % (df2.iloc[0]["nats_rx_time"], df2.iloc[0]["seq"]  ))
        df = pd.concat([df, df2], axis=0)
    return df

ns = await ns_from_time(datetime.datetime.now()- datetime.timedelta(seconds=7200))
df = await nats_stream_fetch(ns, 1)
print("df: %s" % df)

start time 2022-09-02T09:52:57.870134Z
nats options {'servers': ['nats://dbs001.edgefarm.dev:4222'], 'user_credentials': '/home/klaus/work/ops.edgefarm-deployments/overlays/edgefarm.dev/dbs001/source/nsc/nkeys/creds/edgefarm_operator/acc3/user.creds'}
nats_stream_fetch 1
message data: b'{"id":"9297eb90-fe15-4bb0-a1d6-cde5b9ea4b55","source":"bogie-edge","type":"com.dapr.event.sent","traceid":"00-a218dd7e8737380b76513f8a6e09838a-9adb5b64ff9c4e23-00","specversion":"1.0","datacontenttype":"application/octet-stream","topic":"agilis-mc-1.EXPORT.metrics","pubsubname":"bogie-network","tracestate":"","data_base64":"CgwI/aTHmAYQn4i51gISNAoYAAAAAHb8Vz8AAAAAdvxXPwAAACBI+18/EhipJyeeKSpDPw1Ik+k/CkI/62qlJByQRj8aEQgBFUK5RUId5CQxQS1SuB4+IgUNAAAkQioCCAEyBRUAAKZCOhgNPrlFQhXJJDFBJfhT4z0tYOX4QDADOAc="}'
loaded time 2022-09-02 11:53:01.738518 seq 2065
df:                 nats_rx_time   seq                               ts  \
0 2022-09-02 11:53:01.738518  2065 2022-09-02 11:53:01.718160+02:00   

   accel_x_

In [3]:
NUM_SENSORS = 1
class DemoUI:
    def __init__(self, queue, df_cache_size=100):
        self.queue = queue
        self.df_cache_size = df_cache_size
        
        self.start_date_picker = widgets.DatePicker(
            description='Start Date',
            disabled=False
        )
        self.control_box = widgets.Box([self.start_date_picker])

        self.sensor_wf_fig = go.FigureWidget()
        for i in range(NUM_SENSORS):
            self.sensor_wf_fig.add_trace(go.Scatter(x=[], y=[], name="sensor%d" % i))


        self.start_date_picker.observe(self.handle_start_date_change, names='value')

        vbox = widgets.VBox([self.control_box, self.sensor_wf_fig])

        self.df = pd.DataFrame()

        self.draw(0)
        display(vbox)

    def draw(self, idx):
        """
        draw dataframe row at idx
        """
        if idx >= self.df.shape[0]:
            return
        print("draw", idx)
        df = self.df.iloc[idx]
        fig = self.sensor_wf_fig
        
        fig.update_layout({"title": f"Dataset from {df['ts']} seq {df['seq']}"})

        for i in range(NUM_SENSORS):
            fig.data[i].x = df["ts"]
            fig.data[i].y = df["accel_x_rms"]


    async def load_from_time(self, time):
        ns = await ns_from_time(time)
        self.df = await nats_stream_fetch(ns, self.df_cache_size)
        self.slider.value = 0
        self.draw(0)


    def handle_start_date_change(self, change):
        self.queue.put_nowait({"type": "start_date", "value": change.new})


async def loader(queue, ui):
    while True:
        command = await queue.get()
        queue.task_done()
        print(f'loader got {command}')
        if command["type"] == "start_date":
            await ui.load_from_time(command["value"])
        else:
            print("unknown command")

async def main(queue, ui):

    # fire up the both producers and consumers
    asyncio.create_task(loader(queue, ui))

queue = asyncio.Queue()
ui = DemoUI(queue)


#asyncio.run()
await main(queue, ui)


    'data': […

loader got {'type': 'start_date', 'value': datetime.date(2022, 9, 2)}
stime 2022-09-02T00:00:00.000000Z
nats options {'servers': ['nats://dbs001.edgefarm.dev:4222'], 'user_credentials': '/home/klaus/work/ops.edgefarm-deployments/overlays/edgefarm.dev/dbs001/source/nsc/nkeys/creds/edgefarm_operator/acc3/user.creds'}


Task exception was never retrieved
future: <Task finished name='Task-4' coro=<loader() done, defined at /tmp/ipykernel_367952/773267217.py:55> exception=UnicodeDecodeError('utf-8', b'{"source":"bogie-edge","type":"com.dapr.event.sent","pubsubname":"bogie-network","traceid":"00-f96347241789d3e323bad682be6698e5-784e1ab47296dddf-00","id":"23d1d562-226f-4a10-a968-0958b8c43e42","specversion":"1.0","datacontenttype":"application/protobuf","topic":"agilis-mc-1.EXPORT.metrics","tracestate":"","data":"\\n\\u000c\\u0008\xe2\xdf\xc6\x98\\u0006\\u0010\xc7\x94\xc3\x84\\u0001\\u00124\\n\\u0018\\u0000\\u0000\\u0000\\u0000v\xfcW?\\u0000\\u0000\\u0000\\u0000v\xfcW?\\u0000\\u0000\\u0000 H\xfb_?\\u0012\\u0018X\\u0007\xabm\\u0017:D?\x96\xa8\xeb\xc0$\xe9A?<\xcfc\x9d\x84wI?\\u001a\\u0011\\u0008\\u0001\\u0015\\u0015\xb9EB\\u001d\xf6#1A-\xfc\xa9q>\\"\\u0005\\r\\u0000\\u0000\\u001cB*\\u0002\\u0008\\u00012\\u0005\\u0015\\u0000\\u0000\xa2B:\\u0018\\r\\u0016\xb9EB\\u0015\xe9#1A%\\u001dZd>-\\u001f\x85\xeb@0\\u0003

nats_stream_fetch 100
message data: b'{"source":"bogie-edge","type":"com.dapr.event.sent","pubsubname":"bogie-network","traceid":"00-f96347241789d3e323bad682be6698e5-784e1ab47296dddf-00","id":"23d1d562-226f-4a10-a968-0958b8c43e42","specversion":"1.0","datacontenttype":"application/protobuf","topic":"agilis-mc-1.EXPORT.metrics","tracestate":"","data":"\\n\\u000c\\u0008\xe2\xdf\xc6\x98\\u0006\\u0010\xc7\x94\xc3\x84\\u0001\\u00124\\n\\u0018\\u0000\\u0000\\u0000\\u0000v\xfcW?\\u0000\\u0000\\u0000\\u0000v\xfcW?\\u0000\\u0000\\u0000 H\xfb_?\\u0012\\u0018X\\u0007\xabm\\u0017:D?\x96\xa8\xeb\xc0$\xe9A?<\xcfc\x9d\x84wI?\\u001a\\u0011\\u0008\\u0001\\u0015\\u0015\xb9EB\\u001d\xf6#1A-\xfc\xa9q>\\"\\u0005\\r\\u0000\\u0000\\u001cB*\\u0002\\u0008\\u00012\\u0005\\u0015\\u0000\\u0000\xa2B:\\u0018\\r\\u0016\xb9EB\\u0015\xe9#1A%\\u001dZd>-\\u001f\x85\xeb@0\\u00038\\u0007"}'
